In [ ]:
import os

import json

import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertForMaskedLM, BertModel

In [ ]:
WORK_DIR = "/data/disk5/private/yuc/coref/bert-tagger"
FILE_LIST = "filelist.txt"
WIKI_DIR = os.path.join(WORK_DIR, "../wikipedia/text")
# DUMP_DIR =  os.path.join(WORK_DIR, "playground/dump")
DUMP_DIR = os.path.join(WORK_DIR, "playground/dump_kl_para")
LOG_DIR = os.path.join(WORK_DIR, "playground/logs")

In [ ]:
print("Global initialization started.")
global_device = torch.device('cuda')
global_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
global_model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(global_device)
print("Global initialization completed.")

In [ ]:
def default_transform(sentence, device=global_device):
    if sentence[0] == "<":
        return None
    # raw_tokens: parsed into subword but not yet converted to ids
    raw_tokens = global_tokenizer.tokenize(sentence)
    # tokens converted to ids
    tokens = global_tokenizer(sentence, return_tensors="pt")["input_ids"].to(device)
    tokens = torch.squeeze(tokens)
    # l = len(tokens)
    l = tokens.shape[0]
    if l >= 30: # ignore doc that is too long
        return None
    if l <= 5: # ignore invalid lines and short sentences
        return None

    # this is the format of a sentence.
    return {
        "tokens": tokens,
        "raw": raw_tokens
    }

sentence = global_tokenizer.convert_tokens_to_string(['cochran', '–', 'arm', '##ita', '##ge', 'test', 'for', 'trend'])
print(sentence)
ts = default_transform(sentence)
print(ts)

In [ ]:
class SentenceIterable:
    def __init__(self,
        file_path_list=FILE_LIST,
        file_id=0,
        stc_id=0,
        transform=default_transform):
        self.file_id = file_id
        self.stc_id = stc_id
        with open(file_path_list, "r") as f_list:
            self.file_paths = f_list.read().split()
        if transform == None:
            self.transform = default_transform
        else:
            self.transform = transform
        print("SentenceIterable constructed.")
        
    def __iter__(self):
        return self.sentence_generator()
    
    def sentence_generator(self):
        file_count = len(self.file_paths)
        while self.file_id < file_count:
            file_path = self.file_paths[self.file_id]
            with open(file_path) as fs:
                sentences = fs.readlines()
                sentence_count = len(sentences)
                while self.stc_id < sentence_count:
                    sentence = sentences[self.stc_id]
                    sentence = self.transform(sentence)
                    if sentence == None:
                        print("sentence discarded.")
                    else:
                        yield (sentence, self.file_id, self.stc_id)
                    self.stc_id += 1
            self.stc_id = 0
            self.file_id += 1

In [ ]:
class QuestionPairIterable(Dataset):
    def __init__(self, 
        sentence,
        mask_placeholder="[MASK]",
        miss_placeholder="[MASK]"):
        super(QuestionPairIterable).__init__()
        self.sentence = sentence["tokens"]
        self.miss_ph = miss_placeholder
        self.mask_ph = mask_placeholder
        self.miss_id = global_tokenizer.convert_tokens_to_ids(miss_placeholder)
        self.mask_id = global_tokenizer.convert_tokens_to_ids(mask_placeholder)
        length = len(self.sentence)
        self.index_pairs = [
            ([miss_index], [mask_index])
            for miss_index in range(1, length-1)
                for mask_index in range(1, length-1)
                    if miss_index != mask_index
        ]

        self.start = 0
        self.end = len(self.index_pairs)
        print("QuestionPairIterable constructed.")

    def __len__(self):
        return len(self.index_pairs)

    def __getitem__(self, index):
        missing_indices, masked_indices = self.index_pairs[index]
        # unmasked_question = list(self.sentence)
        unmasked_question = self.sentence.clone()
        for missing_index in missing_indices:
            # unmasked_question[missing_index] = self.miss_ph
            unmasked_question[missing_index] = self.miss_id
        # masked_question = list(unmasked_question)
        masked_question = unmasked_question.clone()
        for masked_index in masked_indices:
            # masked_question[masked_index] = self.mask_ph
            masked_question[masked_index] = self.mask_id
        return {
            "label": self.sentence,
            "unmasked": unmasked_question, 
            "masked": masked_question, 
            "miss_id": torch.tensor(missing_indices), 
            "mask_id": torch.tensor(masked_indices)
        }
    
def test_pair_iterable():
    sentence = {
        "tokens": torch.from_numpy(np.array([1,2,3,4,5]))
    }
    dataset = QuestionPairIterable(sentence)
    for sample in dataset:
        print(sample)

test_pair_iterable()

In [ ]:
sentence = global_tokenizer.convert_tokens_to_string(['cochran', '–', 'arm', '##ita', '##ge', 'test', 'for', 'trend'])
sentence = default_transform(sentence)
producer = QuestionPairIterable(sentence)
for sample in producer:
    print(sample)

In [ ]:
class QuestionPairConsumer:
    def __init__(self,
        tokenizer=global_tokenizer,
        model=global_model,
        measure=kl_divergence_dist):
        self.tokenizer = tokenizer
        self.model = model
        self.measure = measure
    
    def consume_question_pair(self, question_pair):
        # [B(atch), L(ength of sentence)]
        context = question_pair["label"]
        unmasked = question_pair["unmasked"]
        masked = question_pair["masked"]
        # [B(atch), n(umber of missing tokens)]
        missing_indices = question_pair["miss_id"]
        masked_indices = question_pair["mask_id"]
        # u_pred = consume_question(unmasked, context)
        # m_pred = consume_question(masked, unmasked)
        # [B(atch), L(ength of sentence), V(ocabulary size)]
        u_logits = self.model(input_ids=unmasked).logits
        m_logits = self.model(input_ids=masked).logits

        missing_label_ids = torch.gather(context, 1, missing_indices) # [B, n]
        answer_shape = list(missing_indices.shape)
        answer_shape.append(u_logits.shape[2])
        missing_indices = missing_indices.unsqueeze(2).expand(answer_shape) # [B, n, V]
        missing_label_ids = missing_label_ids.unsqueeze(2).expand(answer_shape) # [B, n, V]
        
        ones_template = torch.tensor([[[1.]]]).expand(answer_shape) # [B, n, V]
        # golden logits ,g_logits[b][n][index[b][n]] = 1
        g_logits = torch.scatter(torch.zeros(answer_shape), 2, missing_label_ids, ones_template)
        # unmasked logits
        u_logits = torch.gather(u_logits, 1, missing_indices)
        # masked logits
        m_logits = torch.gather(m_logits, 1, missing_indices)
        
        return self.measure(m_logits, u_logits, g_logits)

In [ ]:
def get_batch_size(batch):
    for value in batch.values():
        return value.shape[0]

class SaveManager:
    def __init__(self,
        dump_dir=DUMP_DIR,
        counter=0,
        log_interval=100,
        save_interval=500):
        self.sentence_dict = {}
        self.relation_list = []
        self.log_interval = log_interval
        self.save_interval = save_interval
        self.counter = counter - counter % save_interval
        self.dump_dir = dump_dir
        self.progress_path = os.path.join(self.dump_dir, "progress.log")
        self.rel_template = os.path.join(dump_dir, "relation_list_cnt_{}.dump")
        self.stc_template = os.path.join(dump_dir, "sentence_dict_cnt_{}.dump")

    def load_progress(self):
        return (0, 0)

        if os.path.exists(self.progress_path):
            with open(self.progress_path, "r") as p_log:
                progress = json.load(p_log)
                file_id = progress["file_id"]
                stc_id = progress["stc_id"]
                self.save_interval = progress["save_interval"]
                self.counter = progress["counter"]
                return (file_id, stc_id)
        return (0, 0)

    def dump_progress(self, file_id, stc_id):
        with open(self.progress_path, "w") as p_log:
            progress = {
                "file_id": file_id,
                "stc_id": stc_id,
                "counter": self.counter,
                "save_interval": self.save_interval
            }
            p_log.write(json.dumps(progress))
        
    def save_sentence_list(self):
        sentence_list = []
        for context_id, raw_tokens in self.sentence_dict:
            self.sentence_list.append({
            "id": context_id,
            "context": raw_tokens
        })
        sentence_list.sort(key=lambda x:x["id"])
        save_path = self.stc_template.format(self.counter)
        with open(save_path, "w") as f:
            for sentence in sentence_list:
                f.write(json.dumps(sentence)+"\n")

    def update_sentence(self, sentence, context_id):
        self.sentence_dict[context_id] = sentence["raw"]
    
    def save_relation_list(self):
        save_path = self.rel_template.format(self.counter)
        with open(save_path, "w") as f:
            for relation in self.relation_list:
                f.write(json.dumps(relation)+"\n")
    
    def update_relation(self, sample, distance, context_id):
        self.relation_list.append({
            "context": context_id,
            "missing_index": sample["miss_id"].tolist(),
            "masked_index": sample["mask_id"].tolist(),
            "distance": float(distance)
        })
        self.counter += 1
        if self.counter % self.log_interval == 0:
            print("Got example count: ", self.counter)
        if self.counter % self.save_interval == 0:
            print("Save examples.")
            save_relation_list()
            save_sentence_list()
            self.relation_list = []
            self.sentence_dict = {}

    def update_relation_batched(self, batch, distance, context_id):
        batch_size = get_batch_size(batch)
        for index in range(0, batch_size):
            relation = {}
            for key, batched_tensor in batch.items():
                relation[key] = batched_tensor[index]
            self.update_relation(relation, distance[index], context_id)

In [ ]:
def main():
    sentence_list = []
    relation_list = []
    log_interval = 100
    save_interval = 500

    save_manager = SaveManager(save_interval=save_interval)
    last_file_id, last_stc_id = save_manager.load_progress()
   
    sentence_dataset = SentenceIterable(file_id=last_file_id,stc_id=last_stc_id)
    consumer = QuestionPairConsumer() 

    for sentence, file_id, stc_id in sentence_dataset:
        context_id = file_id * 50000 + stc_id
        stc_relation_list = []
        question_pair_dataset = QuestionPairIterable(sentence)
        dataloader = DataLoader(question_pair_dataset, batch_size=32, num_workers=0)
        for sample_batched in dataloader:
            distance = consumer.consume_question_pair(sample_batched)
            save_manager.update_sentence(sentence, context_id)
            save_manager.update_relation_batched(sample_batched, distance, context_id)
            save_manager.dump_progress(file_id, stc_id)
            break
        break

In [ ]:
main()

In [ ]:
sample = main()
label = sample["label"]
unmasked = sample["unmasked"]
masked = sample["masked"]
print(sample)

In [ ]:
consumer = QuestionPairConsumer()
g_logits, u_logits, m_logits = consumer.consume_question_pair(sample)
print(g_logits, u_logits, m_logits)

In [ ]:
# result: masked; target: unmasked index: golden
def index_only_dist(result, target, index):
    n_dim = 1
    v_dim = 2
    return torch.mean(
        torch.sum(
            F.relu(target - result) * index, dim=v_dim
        ),
        dim=n_dim
    )

def kl_divergence_dist(result, target, index):
    n_dim = 1
    v_dim = 2
    return torch.mean(
        torch.sum(
            F.softmax(target, dim=v_dim) * ( - F.log_softmax(result, dim=v_dim) + 
            F.log_softmax(target, dim=v_dim)),
            dim=v_dim
        ),
        dim=n_dim
    )

def cross_entropy_dist(result, target, index):
    n_dim = 1
    v_dim = 2
    return torch.mean(
        torch.sum(
            - F.softmax(target, dim=v_dim) * F.log_softmax(result, dim=v_dim),
            dim=v_dim
        ),
        dim=n_dim
    )

print(index_only_dist(u_logits, m_logits, g_logits))
print(kl_divergence_dist(u_logits, m_logits, g_logits))
print(cross_entropy_dist(u_logits, m_logits, g_logits))

In [1]:
hamlet = "To be or not to be, that is the question."
hysteria = "a psychoneurosis marked by emotional excitability and disturbances of " \
    "the psychogenic, sensory, vasomotor, and visceral functions"
village = "The statistical area Berkhout which also can include the peripheral " \
    "parts of the village as well as the surrounding countryside has a population of around 1780."

In [ ]:
tokenizer_to_train = BertTokenizer.from_pretrained('bert-base-uncased')
model_to_train = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
inputs = tokenizer_to_train(hamlet, return_tensors="pt")
outputs = model_to_train(**inputs)

In [ ]:
print(outputs.keys())

In [ ]:
import torch
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.train()
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [ ]:
labels = torch.tensor([1,0]).unsqueeze(0)
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss
loss.backward()
optimizer.step()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
To be or not to be, that is the question.


In [5]:
doc = nlp(hysteria)
print(doc)
for np in doc.noun_chunks:
    print(np.start, np.end)

a psychoneurosis marked by emotional excitability and disturbances of the psychogenic, sensory, vasomotor, and visceral functions
0 2
4 6
7 8
9 19


In [10]:
for token in doc:
    print(type(token))
    break

<class 'spacy.tokens.token.Token'>
